In [1]:
# importazione librerie utili
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from boruta import BorutaPy 
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score, matthews_corrcoef, cohen_kappa_score
from imblearn.over_sampling import SMOTENC

In [2]:
#Funzione per calcolare la specificità TrueNeg/TotSani

def specificity_score(ypred,y):
     tn, fp, fn, tp = confusion_matrix(ypred, y).ravel()
    
     return (tn/(tn+fp))

In [3]:
#Funzione per calcolare la media
def media (array, int):
    c=0
    for i in array:
        c=c+i
    return (c/int)

In [4]:
def augmentation(X_train_selected, y_train, cat_indexes):
    sm=SMOTENC(random_state=41, categorical_features=cat_indexes)
    X_train_selected_aug, y_train_aug = sm.fit_resample(X_train_selected, y_train)
    return X_train_selected_aug, y_train_aug

In [5]:
def apply_boruta(X_train, y_train, X_test, colname, rf):
    boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=0, max_iter = 50, perc = 40)
    
    #modello che prende in entrata x e la loro corrispondete uscita y e dice quali features sono rilevanti e quali no.
    boruta_feature_selector.fit(X_train, y_train)
    indexes = np.where(boruta_feature_selector.support_ == True)
    #print(colname[indexes])
    #quindi qui ho 270x20 e 30x20        
    X_train_selected=boruta_feature_selector.transform(X_train)
    #X_train_selected=X_train[:,indexes]
    X_test_selected=boruta_feature_selector.transform(X_test)
    
    return indexes, X_train_selected, X_test_selected 

In [6]:
def apply_rfe(X_train,y_train, X_test,colname, rf):
    selector = RFE(rf, n_features_to_select=20, step=3)
    selector = selector.fit(X_train, y_train)
              
    X_train_selected=selector.transform(X_train)
    X_test_selected=selector.transform(X_test)
    indexes = np.where(selector.support_ == True) 
    #print(colname[indexes])
    
    return indexes,X_train_selected,X_test_selected 

In [7]:
#TUTTE LE VARIABILI
#carico il dataset
data = pd.read_csv(r'C:\Users\Utente\anaconda3\Lib\site-packages\pandas\io\data_covnet_score-imputed_missRF_increasing_1.txt')
print(data.shape)

# fisso il numero di iterazioni di cross-validation
nit=15

# fisso il numero di fold della cross vvalidation
n_fold=10
#creo un array di sole features
features = np.array([f for f in data.columns if f not in ['LABEL']])

#print(categorico)
#creo un array con solo i valori delle features.
X = data[features].values

#ravel() mi fa diventare l'array monodimensionale.
Y = data['LABEL'].values.ravel()

#definisco una random forest
rf = RandomForestClassifier(n_estimators = 101, max_depth=11, max_features=None, max_samples=0.75,class_weight="balanced")


#creo un array monodimensionale lungo quanto "altri" ma solo di 0. 
y_pred=np.zeros(Y.shape[0])
ytrue = np.zeros(Y.shape[0])
conta =np.zeros(Y.shape[0])

#creo degli array che mi servono per creare i grafici, dove andrò a salvare le features che sono selezionate
parzial_featuresboruta = []
final_featuresboruta = list()
parzial_featuresrfe = list()
final_featuresrfe = list()
    
#creo degli array dove salvo di volta in volta il valore degli indici statistici
spec=np.zeros(nit)
sens = np.zeros(nit)
f1=np.zeros(nit)
acc=np.zeros(nit)
prec=np.zeros(nit)
matt =np.zeros(nit)
choen=np.zeros(nit)    


#DATA AUGMENTATION ?
data_augmentation = True #False

    
#file_testo = open('prova.txt','w')
#x = 2
#y = 3
#file_testo.write('Questa è una prova di scrivere 10 %f %f' %(x,y)) 

#print(X_train_selected_aug.shape) 500 22

#file_testo.close()



(301, 42)


In [8]:
def boruta():
    
    for niter in range(nit):
        skf = StratifiedKFold(n_splits=n_fold)

        for train_index, test_index in skf.split(X, Y):
               
                # mi creo i punti di train (270) e di test (30) ad ogni iterazione cambiano 
                X_train, X_test = X[train_index,:], X[test_index,:]
                #creo le label del train e quelle del test (monodimensionale lungo 270 e 30)
                y_train, y_test = Y[train_index], Y[test_index]

                #BORUTA
               
                indexes, X_train_selected, X_test_selected = apply_boruta(X_train, y_train, X_test,data.columns, rf)
                indici=[]
                for x in np.nditer(indexes):
                    parzial_featuresboruta.append(features[x])
                    indici.append(int(x))
                    print(x)
                    
                if data_augmentation:
                    cat_indexes = []
                    #divido in categorici e numerici. i lo uso come indice, feature mi indica la i-esima feature 
                    print(indici)
                    features_selected = features[indici] #Da 42 a 20
                    for i, feature in enumerate(features_selected): #Itero con indice e feature solo sulle selezionate
                        if feature.startswith('CAT'):
                            cat_indexes.append(i)
                            print(cat_indexes)


                    X_train_selected_aug, y_train_aug = augmentation(X_train_selected, y_train, cat_indexes)
                    clf = rf.fit(X_train_selected_aug, y_train_aug)
                else:
                    clf = rf.fit(X_train_selected, y_train)
              
                #y_pred=array di 300x1 0. Lo riempio di 30 alla volta andando a mettere la predizione basata su x_test_selected
                y_pred[test_index] = clf.predict(X_test_selected)
               




        #fine ciclo for train_index, test_index in skf.split(X, Y):

        # QUI ho le predizioni ottenute con questpo split del dataset
        #mi salvo il valore di ogni indice statistico, nell'array. Dovrò farne la media.
        tp, tn, fp, fn = confusion_matrix(y_pred, Y).ravel()
        f1[niter]=f1_score(y_pred,Y)
        sens[niter] = recall_score(y_pred,Y)
        acc[niter]=accuracy_score(y_pred,Y)
        prec[niter]=precision_score(y_pred,Y)
        matt[niter]=matthews_corrcoef(y_pred,Y)
        choen[niter]=cohen_kappa_score(y_pred,Y)
        spec[niter]=specificity_score(y_pred,Y)

        #fine for niter in range(nit) -> 

   # X_filtered = boruta_feature_selector.transform(X)
    #indexes = np.where(boruta_feature_selector.support_ == True)
  

    #stampo la media di tutti gli indici statistici per Boruta.
    print("Specificità: ",media(spec,nit),"F_score: ",media(f1,nit), " Sens: ",media(sens,nit)," Accuratezza: ", media(acc,nit), " Precision: ",media(prec,nit), " Matthews: ",media(matt,nit), " Choen Kappa: ", media(choen, nit))




In [9]:
def rfe():

    for niter in range(nit):
        skf = StratifiedKFold(n_splits=n_fold)

        for train_index, test_index in skf.split(X, Y):
            
                # mi creo i punti di train e di test
                X_train, X_test = X[train_index,:], X[test_index,:]
                #creo le label del train e quelle del test
                y_train, y_test = Y[train_index], Y[test_index]

                
                #RFE
                
                indexes, X_train_selected, X_test_selected=apply_rfe(X_train,y_train, X_test,data.columns, rf)


                for x in np.nditer(indexes):
                    parzial_featuresrfe.append(features[x])

                clf = rf.fit(X_train_selected, y_train)

                y_pred[test_index] =clf.predict(X_test_selected)
             
        #fine ciclo for train_index, test_index in skf.split(X, Y):

        # QUI ho le predizioni ottenute con questo split del dataset
        #mi salvo il valore di ogni indice statistico, nell'array. Dovrò farne la media.
        tp, tn, fp, fn = confusion_matrix(y_pred, Y).ravel()
        f1[niter]=f1_score(y_pred,Y)
        sens[niter] = recall_score(y_pred,Y)
        acc[niter]=accuracy_score(y_pred,Y)
        prec[niter]=precision_score(y_pred,Y)
        matt[niter]=matthews_corrcoef(y_pred,Y)
        choen[niter]=cohen_kappa_score(y_pred,Y)
        spec[niter]=specificity_score(y_pred,Y)

    #fine for niter in range(nit) -> 

    #X_filtered = selector.transform(X)
    #indexes = np.where(selector.support_ == True)
    #stampo la media di tutti gli indici statistici per RFE.
    print("Specificità: ",media(spec,nit),"F_score: ",media(f1,nit), " Sens: ",media(sens,nit)," Accuratezza: ", media(acc,nit), " Precision: ",media(prec,nit), " Matthews: ",media(matt,nit), " Choen Kappa: ", media(choen, nit))




In [10]:
boruta()
print()

#creo un array lungo quanto le features in cui andrò a salvare il numero di volte in cui una features è stata selezionata
contaboruta=[0]*41


#creo degli array per salvarmi quante volte una features è stata selezionata, in base a una divisione del dataset,
#così riuscirò a ottenere dei grafici ridotti e sensati
primadivisioneboruta=[0]*14 #da CAT.Fever a INT.No.Symptoms + INT.Symptoms.No.days--> 26esimaposizione (quindi25)0-12+25
secondadivisioneboruta=[0]*10 #da CAT.Pneumo.asthma a INT.No.Comorbidities 13-22
terzadivisioneboruta=[0]*8 #23-24-26-31 no 25 CAT.Sex,INT.Age
quartadivisioneboruta=[0]*9  #INT.ALT-NUM.Haematocrit 32-40

#Le variabili sotto mi servono per tener conto fin dove sono arrivato a riempire i vari array.
a=0
primo=0
secondo=0
terzo=0
quarto=0

for i in features:
  c=parzial_featuresboruta.count(i)
  contaboruta[a]=c  
#in base alle divisioni del dataset fatte sopra riempio gli array, a seconda del valore di a.
  if (a<=12 or a==25):
    primadivisioneboruta[primo]=c
    primo=primo+1
  if(a>=13 and a<=22):
    secondadivisioneboruta[secondo]=c
    secondo=secondo+1
  if(a>=23 and a<=31 and a!=25):
    terzadivisioneboruta[terzo]=c
    terzo=terzo+1
  if (a>=32 and a<=40):
    quartadivisioneboruta[quarto]=c
    quarto=quarto+1
  print(i,'        ', c, c/(nit*n_fold))
  a=a+1  
  if (c>90):
    final_featuresboruta.append(i)
print(final_featuresboruta)
print(contaboruta)

8
18
22
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 18, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
[0, 1]
8
12
19
22
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 12, 19, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
[0, 2]
8
12
15
22
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 12, 15, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
[0, 2]
8
12
15
22
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 12, 15, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
[0, 2]
8
19
22
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 19, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
[0, 1]
8
12
22
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[8, 12, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[0]
9
12
22
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[9, 12, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
rfe()
print()
#creo un array lungo qunto le features in cui andrò a salvare il numero di volte in cui una features è stata selezionata
contarfe=[0]*41

#creo degli array per salvarmi quante volte una features è stata selezionata, in base a una divisione del dataset,
#così riuscirò a ottenere dei grafici ridotti e sensati

primadivisionerfe=[0]*14 #da CAT.Fever a INT.No.Symptoms + INT.Symptoms.No.days--> 26esimaposizione (quindi25)0-12+25
secondadivisionerfe=[0]*10 #da CAT.Pneumo.asthma a INT.No.Comorbidities 13-22
terzadivisionerfe=[0]*8 #23-24-26-31 no 25 CAT.Sex,INT.Age
quartadivisionerfe=[0]*9  #INT.ALT-NUM.Haematocrit 32-40

#creo degli array per salvarmi il nome delle features, secondo la divisione del dataset che voglio rappresentare graficamente
feat1=[0]*14
feat2=[0]*10
feat3=[0]*8
feat4=[0]*9

#Le variabili sotto mi servono per tener conto fin dove sono arrivato a riempire i vari array.
a=0
primo=0
secondo=0
terzo=0
quarto=0

for i in features:
  c=parzial_featuresrfe.count(i)
  contarfe[a]=c  
    #in base alle divisioni del dataset fatte sopra riempio gli array, a seconda del valore di a.
  if (a<=12 or a==25):
    primadivisionerfe[primo]=c
    feat1[primo]=i
    primo=primo+1
    
  if(a>=13 and a<=22):
    secondadivisionerfe[secondo]=c
    feat2[secondo]=i
    secondo=secondo+1
    
  if(a>=23 and a<=31 and a!=25):
    terzadivisionerfe[terzo]=c
    feat3[terzo]=i
    terzo=terzo+1
    
  if (a>=32 and a<=40):
    quartadivisionerfe[quarto]=c
    feat4[quarto]=i
    quarto=quarto+1
    
  print(i,'        ', c, c/(nit*n_fold))
  a=a+1  
  if (c>90):
    #ottengo le features finali scelte da RFE
    final_featuresrfe.append(i)
print(final_featuresrfe)
print(contarfe)

In [ ]:
#definisco come variavile per l'asse x 
x=feat1
legend=['Boruta','RFE']
pos = np.arange(len(x))
#imposto lo distanza tra le colonne
bar_width = 0.35
#come y scelgo i dati inerenti a feat1
y1=primadivisioneboruta
y2=primadivisionerfe
plt.figure(figsize=(27,5))
plt.bar(pos,y1,bar_width,color='blue',edgecolor='black')
plt.bar(pos+bar_width,y2,bar_width,color='pink',edgecolor='black')
plt.xticks(pos, x)
#definisco le varie dimensioni per i caratteri delle figure
plt.xlabel('Features', fontsize=16)
plt.ylabel('Volte selezionate', fontsize=16)
plt.title('Features rilevanti Boruta vs RFE',fontsize=18)
plt.legend(legend,loc=2)
plt.show()

In [ ]:
x=feat2
legend=['Boruta','RFE']
pos = np.arange(len(x))
bar_width = 0.3
y1=secondadivisioneboruta
y2=secondadivisionerfe
plt.figure(figsize=(26, 5))
plt.bar(pos,y1,bar_width,color='blue',edgecolor='black')
plt.bar(pos+bar_width,y2,bar_width,color='pink',edgecolor='black')
plt.xticks(pos, x)
plt.xlabel('Features', fontsize=16)
plt.ylabel('Volte selezionate', fontsize=16)
plt.title('Features rilevanti Boruta vs RFE',fontsize=18)
plt.legend(legend,loc=2)
plt.show()

In [ ]:
x=feat1
legend=['Boruta','RFE']
pos = np.arange(len(x))
bar_width = 0.3
y1=primadivisioneboruta
y2=primadivisionerfe
plt.figure(figsize=(26, 5))
plt.bar(pos,y1,bar_width,color='blue',edgecolor='black')
plt.bar(pos+bar_width,y2,bar_width,color='pink',edgecolor='black')
plt.xticks(pos, x)
plt.xlabel('Features', fontsize=16)
plt.ylabel('Volte selezionate', fontsize=16)
plt.title('Features rilevanti Boruta vs RFE',fontsize=18)
plt.legend(legend,loc=2)
plt.show()

In [ ]:
x=feat3
legend=['Boruta','RFE']
pos = np.arange(len(x))
bar_width = 0.3
y1=terzadivisioneboruta
y2=terzadivisionerfe
plt.figure(figsize=(26, 5))
plt.bar(pos,y1,bar_width,color='blue',edgecolor='black')
plt.bar(pos+bar_width,y2,bar_width,color='pink',edgecolor='black')
plt.xticks(pos, x)
plt.xlabel('Features', fontsize=16)
plt.ylabel('Numero di volte selezionate', fontsize=16)
plt.title('Features rilevanti Boruta vs RFE',fontsize=18)
plt.legend(legend,loc=2)
plt.show()

In [ ]:
x=feat4
legend=['Boruta','RFE']
pos = np.arange(len(x))
bar_width = 0.3
y1=quartadivisioneboruta
y2=quartadivisionerfe
plt.figure(figsize=(26, 5))
plt.bar(pos,y1,bar_width,color='blue',edgecolor='black')
plt.bar(pos+bar_width,y2,bar_width,color='pink',edgecolor='black')
plt.xticks(pos, x)
plt.xlabel('Features', fontsize=16)
plt.ylabel('Volte selezionate', fontsize=16)
plt.title('Features rilevanti Boruta vs RFE',fontsize=18)
plt.legend(legend,loc=2)
plt.show()